# Open-Source only RAG for Testing Scenario Creation

- Sentence Transformers as embedding model
- Postgres as vector store
- Llama.cpp as LLM

#### Sentence Transformers

In [1]:
%pip install llama-index-readers-file pymupdf
%pip install llama-index-vector-stores-postgres
%pip install llama-index-embeddings-huggingface
%pip install llama-index-llms-llama-cpp

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# sentence transformers
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en")

/Users/busraoguzoglu/Desktop/Okul/Dersler/test_verification/project/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Llama CPP

In [3]:
%pip install llama-cpp-python

Note: you may need to restart the kernel to use updated packages.


In [4]:
from llama_index.llms.llama_cpp import LlamaCPP

# model_url = "https://huggingface.co/TheBloke/Llama-2-13B-chat-GGML/resolve/main/llama-2-13b-chat.ggmlv3.q4_0.bin"
model_url = "https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF/resolve/main/llama-2-13b-chat.Q4_0.gguf"

llm = LlamaCPP(
    # You can pass in the URL to a GGML model to download it automatically
    model_url=model_url,
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    model_path=None,
    temperature=0.1,
    max_new_tokens=256,
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window=3900,
    # kwargs to pass to __call__()
    generate_kwargs={},
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": 1},
    verbose=True,
)

llama_model_loader: loaded meta data with 19 key-value pairs and 363 tensors from /Users/busraoguzoglu/Library/Caches/llama_index/models/llama-2-13b-chat.Q4_0.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 5120
llama_model_loader: - kv   4:                          llama.block_count u32              = 40
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 13824
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:         

#### Postgres

In [5]:
%pip install psycopg2-binary pgvector asyncpg "sqlalchemy[asyncio]" greenlet

Note: you may need to restart the kernel to use updated packages.


In [6]:
import psycopg2

# Define your database connection parameters
db_name = "rag_db"  # Use the existing database name here
host = "localhost"
password = "password"  # Replace with your actual PostgreSQL password
port = "5432"  # Default PostgreSQL port
user = "myuser"  # Replace with your actual PostgreSQL username

# Connect directly to the 'rag_db' database
conn = psycopg2.connect(
    dbname=db_name,  # Connect directly to 'rag_db'
    host=host,
    password=password,
    port=port,
    user=user,
)
conn.autocommit = True

# Optionally, perform any operations on 'rag_db' using a cursor
with conn.cursor() as c:
    # Drop and create operations are not needed if the database already exists
    # Here you can perform other database setup actions if necessary
    print(f"Connected to {db_name} successfully.")

# Close the connection when done
#conn.close()

Connected to rag_db successfully.


Create table (checks if exist, truncates if already exist and generates again)

In [7]:
from sqlalchemy import create_engine, text
from sqlalchemy.exc import OperationalError
from llama_index.vector_stores.postgres import PGVectorStore

# Connection parameters
db_params = {
    "user": "myuser",      # Replace with your actual username
    "password": "password",  # Replace with your actual password
    "host": "localhost",   # Adjust if your database is hosted elsewhere
    "port": "5432",        # Default PostgreSQL port
    "database": "rag_db"   # Replace with your actual database name
}

# Create SQLAlchemy engine
engine = create_engine(f"postgresql://{db_params['user']}:{db_params['password']}@{db_params['host']}:{db_params['port']}/{db_params['database']}")

# Update table name to include 'data_' prefix
table_name = "data_software_test_table"  # Postgre adds 'data' in front of table_name
embed_dim = 384  # Embedding dimension

try:
    with engine.connect() as conn:
        # Check if the table exists
        result = conn.execute(text(f"""
            SELECT EXISTS (
                SELECT FROM information_schema.tables 
                WHERE table_schema = 'public' AND table_name = :table_name
            );
        """), {"table_name": table_name}).scalar()

        if result:
            print(f"Table '{table_name}' exists. Cleaning it...")
            conn.execute(text(f"TRUNCATE TABLE {table_name};"))
        else:
            print(f"Table '{table_name}' does not exist. Creating it...")

        # Create a new table for vector store
        vector_store = PGVectorStore.from_params(
            database=db_params["database"],
            host=db_params["host"],
            password=db_params["password"],
            port=db_params["port"],
            user=db_params["user"],
            table_name="pmc_table",  # Use unprefixed name; PGVectorStore adds 'data_' automatically
            embed_dim=embed_dim,
        )
        print(f"Table '{table_name}' has been created.")
except OperationalError as e:
    print(f"Error: {e}")
    print("Make sure the database exists and connection parameters are correct.")

Table 'data_software_test_table' does not exist. Creating it...
Table 'data_software_test_table' has been created.


## Ingestion Pipeline

### Data Loading and Preprocessing:

Preprocessing of the document does this:

1- Remove all text after the stopwords, like 'References', because they do not have any information for us.

2- Remove section numbering, letters, extra spaces, new lines.

3- Remove 'tables' from the text.

In [8]:
from pathlib import Path
from llama_index.readers.file import PyMuPDFReader
from llama_index.core.node_parser import SentenceSplitter

## Preprocessing 

### HTML Preprocessing

In [9]:
from bs4 import BeautifulSoup
import re
from pathlib import Path

def extract_text_and_elements(html_file_path):
    """Extract meaningful information from static HTML."""
    # Load the HTML content from the file
    with open(html_file_path, "r", encoding="utf-8") as file:
        html_content = file.read()
    
    # Parse the HTML content
    soup = BeautifulSoup(html_content, "html.parser")
    
    # Extract the title of the page
    title = soup.title.string.strip() if soup.title else "No Title"
    
    # Extract meta description (if available)
    meta_description = ""
    meta_tag = soup.find("meta", attrs={"name": "description"})
    if meta_tag and meta_tag.get("content"):
        meta_description = meta_tag["content"]
    
    # Extract all visible text (paragraphs, headings, etc.)
    text_content = []
    for tag in soup.find_all(["p", "h1", "h2", "h3", "h4", "h5", "h6"]):  # Common text tags
        text = tag.get_text(strip=True)
        if text:
            text_content.append(text)
    
    # Extract all button labels
    buttons = [button.get_text(strip=True) for button in soup.find_all("button")]
    
    # Extract all links (text and URLs)
    links = []
    for link in soup.find_all("a", href=True):
        link_text = link.get_text(strip=True)
        links.append({"text": link_text, "url": link["href"]})
    
    # Structure the extracted information
    extracted_content = {
        "title": title,
        "meta_description": meta_description,
        "text_content": "\n".join(text_content),
        "buttons": buttons,
        "links": links,
    }
    
    return extracted_content

def preprocess_for_rag(content):
    """Preprocess extracted HTML content for use in a RAG system."""
    # Combine title, meta description, text content, and button labels
    combined_content = (
        f"Title: {content['title']}\n\n"
        f"Meta Description: {content['meta_description']}\n\n"
        f"Text Content:\n{content['text_content']}\n\n"
        f"Buttons:\n{', '.join(content['buttons'])}\n\n"
        f"Links:\n"
    )
    # Add links in a readable format
    for link in content["links"]:
        combined_content += f"- {link['text']} ({link['url']})\n"
    
    return combined_content

# Input and Output Paths
html_file_path = Path("./data/imdbhtml.html")  # Replace with your actual HTML file path
output_path = Path("./processed_html.txt")  # File to save processed output

# Extract and preprocess the HTML content
extracted_content = extract_text_and_elements(html_file_path)
processed_content = preprocess_for_rag(extracted_content)

# Write the processed content into an output file
with output_path.open("w", encoding="utf-8") as output_file:
    output_file.write(processed_content)

print(f"Processed HTML content written to {output_path}")

Processed HTML content written to processed_html.txt


### pdf and txt preprocessing

In [10]:
import re
from pathlib import Path
from bs4 import BeautifulSoup  # For HTML parsing
from llama_index.readers.file import PyMuPDFReader

# Define stop keywords
stop_keywords = []

def preprocess_text(text, stop_keywords):
    """Preprocess text by stopping at the earliest occurrence of any keyword, ignoring case."""
    earliest_position = len(text)  # Default to the end of the text
    
    for keyword in stop_keywords:
        pattern = rf"\b{re.escape(keyword)}\b"  # Match exact word boundaries
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            print(f"Keyword '{keyword}' found at position {match.start()}")  # Debugging
            earliest_position = min(earliest_position, match.start())
    
    # Truncate the text at the earliest keyword match
    return text[:earliest_position]

def clean_text(block_text):
    """Clean and process individual blocks of text."""
    block_text = " ".join(block_text.split())  # Remove extra spaces and newlines
    block_text = re.sub(r'^\d+\.\s+', '', block_text)  # Remove section numbering
    block_text = re.sub(r'[^A-Za-z0-9\s,.]', '', block_text)  # Keep only basic punctuation and letters

    # Heuristic: Skip blocks that appear to be tables (e.g., many numbers or special formats)
    if len(block_text) < 10 or re.match(r'Page \d+', block_text):
        return None  # Skip short blocks or page numbers

    # Remove blocks that have a high density of numbers or specific patterns (tables)
    if re.search(r'\d{2,}', block_text) and re.search(r'(\d+\.\d+|\d+%)', block_text):
        return None  # Skip blocks with many numeric values

    # Remove blocks with common table headers like 'Table', 'Mediation Effect', etc.
    if re.search(r'\b(Table|Effect|Mediation|Summary|IV|Mediator|Cont)\b', block_text, re.IGNORECASE):
        return None

    return block_text

In [11]:
# Paths to input files
pdf_file_path = Path("./data/IMDb_Software_Design_Document.pdf")  # Replace with your actual PDF file path
html_file_path = Path("./data/processed_html.txt")  # Replace with your actual HTML file path
output_path = Path("./processed_documents.txt")  # File to save processed documents

In [12]:
documents = []

# Process the PDF file
loader = PyMuPDFReader()
loaded_docs = loader.load(file_path=str(pdf_file_path))
print(f"{pdf_file_path.name}: {len(loaded_docs)} documents loaded")

for doc_idx, doc in enumerate(loaded_docs):
    processed_text = preprocess_text(doc.text, stop_keywords)  # Use `doc.page_content` for text
    
    cleaned_blocks = []
    for block in processed_text.split("\n"):
        cleaned_block = clean_text(block)
        if cleaned_block:
            cleaned_blocks.append(cleaned_block)
    
    final_text = "\n".join(cleaned_blocks)
    
    # Append a dictionary to the `documents` list
    documents.append({
        "source": pdf_file_path.name,
        "content": final_text
    })

# Process the plain text file
with open(html_file_path, "r", encoding="utf-8") as txt_file:
    txt_content = txt_file.read()

# Preprocess the text
processed_text = preprocess_text(txt_content, stop_keywords)

cleaned_blocks = []
for block in processed_text.split("\n"):
    cleaned_block = clean_text(block)
    if cleaned_block:
        cleaned_blocks.append(cleaned_block)

final_text = "\n".join(cleaned_blocks)

# Append the processed text file to the documents list
documents.append({
    "source": html_file_path.name,
    "content": final_text
})

# Write the processed text into the output file
with output_path.open("w", encoding="utf-8") as output_file:
    for idx, doc in enumerate(documents):
        output_file.write(f"Document {idx} from {doc['source']}:\n")
        output_file.write(doc['content'] + "\n")
        output_file.write("="*80 + "\n")  # Separator between documents

print(f"Processed documents written to {output_path}")

IMDb_Software_Design_Document.pdf: 7 documents loaded
Processed documents written to processed_documents.txt


In [13]:
print(documents[2])
print(len(documents))

{'source': 'IMDb_Software_Design_Document.pdf', 'content': 'Requirements\n3.1 Functional Requirements\nComprehensive Search Functionality\nAdvanced keyword and phrase search with fulltext indexing.\nAutosuggestions based on search context and user history.\nMultifaceted filtering options e.g., genre, release year, language.\nPredictive search for incomplete or misspelled queries.\nDetailed Information Pages\nRich metadata for movies, TV shows, and celebrities.\nFeatures such as trailers, image galleries, user reviews, and critic ratings.\nRealtime updates for box office performance and award statistics.\nLinks to external streaming platforms or purchase options.\nUser Account Features\nUser registration, loginlogout, and multifactor authentication.\nPersonalized profiles with privacy settings and activity logs.\nOption to link social media accounts for seamless sharing.\nWatchlist and Favorites Management\nDynamic watchlist creation and management with category support.\nAbility to sha

### Splitting Documents

In [14]:
from llama_index.core.node_parser import SentenceSplitter

text_parser = SentenceSplitter(
    chunk_size=1024,
    # separator=" ",
)

In [15]:
text_chunks = []
# Maintain relationship with source doc index to help inject doc metadata
doc_idxs = []

for doc_idx, doc in enumerate(documents):
    # Use dictionary-style indexing to access 'content'
    cur_text_chunks = text_parser.split_text(doc['content'])
    
    # Optionally print chunk info for debugging
    # print(f"Document {doc_idx} has {len(cur_text_chunks)} chunks")
    
    text_chunks.extend(cur_text_chunks)
    doc_idxs.extend([doc_idx] * len(cur_text_chunks))  # Maintain mapping between chunks and their original document

In [16]:
print(len(text_chunks))

11


### Construct Nodes from Text Chunks

In [17]:
from llama_index.core.schema import TextNode

nodes = []

for idx, text_chunk in enumerate(text_chunks):
    node = TextNode(
        text=text_chunk,
        metadata={
            "source": documents[doc_idxs[idx]]["source"],  # Access 'source' from the dictionary
            "doc_index": doc_idxs[idx]  # Optional: document index
        }
    )
    nodes.append(node)

In [18]:
nodes[1]

TextNode(id_='ffdc019d-ffe7-4b6d-a73d-51ac9120dd6a', embedding=None, metadata={'source': 'IMDb_Software_Design_Document.pdf', 'doc_index': 1}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text='The purpose of this document is to outline the software design for the IMDb web platform.\nIMDb is a widelyused online database that provides comprehensive information about\nmovies, TV shows, actors, production crew, trailers, and usergenerated reviews. This\ndocument serves as a blueprint for the development, maintenance, scalability, and\nintegration of advanced features into the platform.\nThe IMDb web platform allows users to\nSearch for and browse movies, TV shows, celebrity profiles, and production details.\nView detailed information about titles, including trailers, cast, crew, reviews, ratings,\nand box office performance.\nCreate user accounts to rate, review, and discuss titles, create wa

### Generate Embeddings for each Node

Using sentence_transformers

In [19]:
for node in nodes:
    node_embedding = embed_model.get_text_embedding(
        node.get_content(metadata_mode="text")
    )
    node.embedding = node_embedding

In [20]:
print(type(nodes[0].embedding))
print(nodes[0].embedding)
print(len(nodes))

<class 'list'>
[-0.07766719162464142, -0.0283577349036932, 0.0010282271541655064, -0.05490560829639435, 0.020613307133316994, 0.013192079961299896, -0.03248661383986473, 0.015318765304982662, -0.019386041909456253, -0.007195263169705868, 0.04887738078832626, -0.00787668488919735, 0.02847866900265217, -0.008579934015870094, -0.0029523735865950584, 0.03200364112854004, 0.013738002628087997, -0.027684304863214493, -0.004265206400305033, 0.044468168169260025, 0.028138332068920135, -0.021146930754184723, -0.012262994423508644, -0.024859324097633362, 0.00979662872850895, 0.0599738210439682, -0.03414741903543472, -0.01959979720413685, -0.045881956815719604, -0.19673268496990204, -0.010792595334351063, -0.017157170921564102, 0.028402989730238914, 0.026830574497580528, -0.023383865132927895, -0.01780080981552601, 0.0018556859577074647, 0.02545739896595478, -0.04630320891737938, -0.00709082605317235, -0.001693759928457439, 0.017659682780504227, 0.005670623388141394, -0.07479432970285416, 0.01115

### Load Nodes into a Vector Store

We now insert these nodes into our `PostgresVectorStore`.

In [21]:
vector_store.add(nodes)

['4baae04c-fac3-451b-a7ea-56aaa4e4bfcc',
 'ffdc019d-ffe7-4b6d-a73d-51ac9120dd6a',
 'edd8b223-7584-46b3-97ee-07471b18bed6',
 'd89f1d60-d80f-4880-b46d-49834cd040a6',
 '0095e6df-3c42-4862-9210-3ca83da201f6',
 'e6dece65-df31-4184-9a3d-c5506ba20738',
 'ecdffd55-b798-449b-92ba-5c28fa978613',
 '2f42ce8a-4172-4f95-bc09-8bb286bc53dd',
 '83ecc003-84b4-44b1-b1fe-aeb713ab9f6d',
 '787228de-c750-429e-918f-a5e4d443dc1d',
 '4f74ade3-9b33-483f-8bf2-b01fa208647d']

## Retrieval Pipeline

In [22]:
from llama_index.core import QueryBundle
from llama_index.core.retrievers import BaseRetriever
from typing import Any, List
from llama_index.core.schema import NodeWithScore
from typing import Optional
from llama_index.core.vector_stores import VectorStoreQuery

In [23]:
class VectorDBRetriever(BaseRetriever):
    """Retriever over a postgres vector store."""

    def __init__(
        self,
        vector_store: PGVectorStore,
        embed_model: Any,
        query_mode: str = "default",
        similarity_top_k: int = 2,
    ) -> None:
        """Init params."""
        self._vector_store = vector_store
        self._embed_model = embed_model
        self._query_mode = query_mode
        self._similarity_top_k = similarity_top_k
        super().__init__()

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        """Retrieve."""
        query_embedding = embed_model.get_query_embedding(
            query_bundle.query_str
        )
        vector_store_query = VectorStoreQuery(
            query_embedding=query_embedding,
            similarity_top_k=self._similarity_top_k,
            mode=self._query_mode,
        )
        query_result = vector_store.query(vector_store_query)

        nodes_with_scores = []
        for index, node in enumerate(query_result.nodes):
            score: Optional[float] = None
            if query_result.similarities is not None:
                score = query_result.similarities[index]
            nodes_with_scores.append(NodeWithScore(node=node, score=score))

        return nodes_with_scores

In [24]:
retriever = VectorDBRetriever(
    vector_store, embed_model, query_mode="default", similarity_top_k=2
)

retriever

## Response

In [25]:
from llama_index.core.query_engine import RetrieverQueryEngine

query_engine = RetrieverQueryEngine.from_args(retriever, llm=llm)

In [26]:
query_str = "What can be a test scenario in IMDB website about login page?"

response = query_engine.query(query_str)


llama_print_timings:        load time =    8644.06 ms
llama_print_timings:      sample time =       9.65 ms /   256 runs   (    0.04 ms per token, 26517.51 tokens per second)
llama_print_timings: prompt eval time =    8643.36 ms /   388 tokens (   22.28 ms per token,    44.89 tokens per second)
llama_print_timings:        eval time =   32689.59 ms /   255 runs   (  128.19 ms per token,     7.80 tokens per second)
llama_print_timings:       total time =   41474.99 ms /   643 tokens


In [27]:
print(str(response))



Based on the provided context information, here are some possible test scenarios for the login page of the IMDB website:

1. Successful login: Test that a valid user can successfully log in to their account using their email address and password.
2. Incorrect login credentials: Test that an invalid email address or password results in an error message.
3. Forgotten password: Test that a user can reset their password using the "Forgot password" feature.
4. Register new user: Test that a new user can successfully register and log in to their account.
5. User account lockout: Test that the system can lock out a user after multiple failed login attempts.
6. User account deletion: Test that a user's account can be deleted by an administrator.
7. User account update: Test that a user's account information can be updated.
8. Password reset: Test that a user can reset their password using the "Forgot password" feature.
9. Email verification: Test that the system sends a verification email to

In [28]:
print(response.source_nodes[0].get_content())

1.4 References
 IMDb official website httpswww.imdb.com
 REST API design principles
 OWASP security guidelines
 2. System Overview
The IMDb web platform is a threetiered architecture consisting of
Frontend A responsive web interface for user interaction.
Backend A scalable service layer handling business logic and APIs.
Database A relational and nonrelational hybrid system for storing structured and
unstructured data.
 3. Requirements
 3.1 Functional Requirements
Search Functionality Provide robust search with autosuggestions for movies, TV shows, and
Detailed Pages Display detailed information for titles, actors, and crew.
User Accounts Enable user registration, login, and profile management.
Watchlist Allow users to add titles to personal watchlists.


In [29]:
query_str = "Generate different test scenarios for IMDB web page"

response = query_engine.query(query_str)

Llama.generate: 29 prefix-match hit, remaining 242 prompt tokens to eval

llama_print_timings:        load time =    8644.06 ms
llama_print_timings:      sample time =       9.66 ms /   256 runs   (    0.04 ms per token, 26503.78 tokens per second)
llama_print_timings: prompt eval time =    5755.44 ms /   242 tokens (   23.78 ms per token,    42.05 tokens per second)
llama_print_timings:        eval time =   31003.18 ms /   255 runs   (  121.58 ms per token,     8.22 tokens per second)
llama_print_timings:       total time =   36901.56 ms /   497 tokens


In [30]:
print(str(response))



Test Scenario 1: Search Functionality

* Enter a random keyword in the search bar
* Click on the search button
* Verify that the search results display a list of relevant movies, TV shows, and celebrities
* Test for different search queries and verify that the results are accurate and relevant

Test Scenario 2: Movie/TV Show Details Page

* Navigate to a specific movie or TV show details page
* Verify that the page displays information such as title, release date, cast, crew, and plot summary
* Test for different types of movies and TV shows and verify that the information is accurate and consistent

Test Scenario 3: User Profile and Watchlist

* Create a user profile and add movies/TV shows to the watchlist
* Verify that the watchlist is displayed correctly and that the user can remove items from the watchlist
* Test for different user actions and verify that the system responds correctly

Test Scenario 4: Movie/TV Show Ratings and Reviews

* Rate a movie or TV show and leave a revi

In [31]:
print(response.source_nodes[0].get_content())

Software Design Document for IMDb
Web Platform
Version 2.0
Prepared by Your Name
Date Insert Date
Introduction
References
System Overview
Requirements
Functional Requirements
NonFunctional Requirements
System Architecture
Detailed Design
Frontend Design
Backend Design
Database Design
ThirdParty Integrations
Security Considerations
Performance Optimization
Testing and Validation
Deployment Strategy
Future Enhancements
Introduction
1.1 Purpose


In [34]:
query_str = "Generate a test flow for User Profile and Watchlist features in IMDB website, explain step by step"

response = query_engine.query(query_str)

Llama.generate: 539 prefix-match hit, remaining 230 prompt tokens to eval

llama_print_timings:        load time =    8644.06 ms
llama_print_timings:      sample time =       9.62 ms /   256 runs   (    0.04 ms per token, 26616.76 tokens per second)
llama_print_timings: prompt eval time =    5852.05 ms /   230 tokens (   25.44 ms per token,    39.30 tokens per second)
llama_print_timings:        eval time =   34575.90 ms /   255 runs   (  135.59 ms per token,     7.38 tokens per second)
llama_print_timings:       total time =   40572.20 ms /   485 tokens


In [35]:
print(str(response))


Step 1: User Registration
* Go to the IMDb website and click on the "Sign Up" button.
* Enter the required information such as email address, username, and password.
* Click on the "Create Account" button.

Step 2: Login
* Go to the IMDb website and click on the "Login" button.
* Enter the registered email address and password.
* Click on the "Login" button.

Step 3: Profile Creation
* After successful login, the user should be redirected to their profile page.
* The profile page should display the user's name, profile picture, and watchlist.
* The user should be able to edit their profile information, including their name, profile picture, and bio.

Step 4: Watchlist Creation
* The user should be able to add movies and TV shows to their watchlist by clicking on the "Add to Watchlist" button.
* The watchlist should be displayed on the user's profile page, and the user should be able to edit the list.
* The user should also be able to remove titles from their watchlist.

Step 5: Search